# Class 6

# Seguimos con el EDA
* Un BoxPlot dice lo mismo que un gráfico de una distribución! (ver Slides!)

## Tratamiento de Outliers

Puede haber outliers POR FILA, es más difícil pero se puede hacer con algorítmos como dbscan!

(reparar la clase4)

## Tratamiento de Missing

* Si son muy muchos en la variable, la eliminamos, tipo 98% missing chau...
* Si vamos a mantener la variable, debemos SUSTITUÍR los valores Missing
    - Si la variable es Categórica, los Missings pueden si representar algo! Muchas veces se rellena con una categoría 'No value', o 'Sin valor'. Si veo que NO influyen en nada, alomejor se podría reemplazar por la moda de las cats, hay que ver!
    - Si la var el Numerical, habría que ver si lo rellenamos por la media, mediana, max o min, o por algún valor que sepamos que representa el missing, como -99 en edad o -1, depende...
        - Estos valores extremos pueden servir en algorítmos de clasificación, como los Trees, en ese caso va bien pq los va a cortar solos
        - Pero OJO, que en una regresión o algo si puede afectar! En vez de edad -99 puede ser -1, hay que ver el tratamiento que se les da...

* ANTES de sacar cualq cosa, hay que ver que relación tienen con la variable Target!

## Tratamiento de correlaciones

* Correlación: Que varíen en proporciones muy similares, ya sea positiva o negativa!
* Solo se pueden aplicar las corr de pearson a variables CONTINUAS (NO CATEGÓRICAS, por más q sean numéricas!)
* En REGRESIONES sobre todo, cualq variable correlacionada HAY QUE ELIMINAR HASTA QUEDARSE SOLO CON UNA MÁS REPRESENTATIVA.
* Corr mayor a 0.75, nos quedamos solo con una!

#### PARA LA PRÁCTICA PONER TODAS LAS CORRELACIONES Y COMENTARLAS!

# MUY IMPORTANTE EN LA PRÁCTICA!!! QUE SEA ESTRATIFICADO EL TRAIN_TEST_SPLIT!!!
# PARA QUE MANTENGAN LAS PROPORCIONES (PORCENTAJES DE 0 Y 1) EN LA VARIABLE OBJETIVO!!!!!!!!!!!!!!!
# ESTO SIEMPRE ES MUY MUCHO MUY IMPORTANTE


```python
from sklearn.model_selection import train_test_split
X_pd_loan, X_pd_loan_test, y_pd_loan, y_pd_loan_test = train_test_split(pd_loan.drop('loan_status',axis=1), 
                                                                     pd_loan['loan_status'], 
                                                                     stratify=pd_loan['loan_status'], 
                                                                     test_size=0.2)
```
Como vemos, stratify=df['VARIABLE TARGET!']


### Sumar nros por filas y columnas:

```python
pd_series_null_columns = pd_loan_train.isnull().sum().sort_values(ascending=False)
pd_series_null_rows = pd_loan_train.isnull().sum(axis=1).sort_values(ascending=False)
print(pd_series_null_columns.shape, pd_series_null_rows.shape)
```

#### El gráfico doble que analiza cada variable es MUY CLAVE, es genial! Muestra distribución y boxplot según los valores del Target!!!

## Tratamiento de variables Continuas VS Categóricas

* Llegado el momento las vamos a separar por varias cosas:
    - Las numéricas pueden reemplazar sus outliers parecido entre ellas, categóricas NO, hay que ver cada caso y q categoría poner...
    - Para las numericas puede sacarse la correlacion
    - hay más...

Formar de separarlas: por el unique(), las categóricas deberían tener pocas y numéricas muchas (ver funciñon)

* Hay que ver si la distribución de la variable Target en Outliers se mantiene o no! Osea, por cada variable, ver cantidad de outliers y ver q distribución de 1 y 0 tienen de la variable objetivo!

* El % de la fórmula para determinar Outliers es MODIFICABLE, no es siempre 1.5 


### Muy bueno. Saca LAS VARIABLES MÁS CORRELACIONADAS entre sí! Esto puede servir para ordernar el HeatMap tmb! CLAVE:
```python
corr = pd_loan_train[list_var_continuous].corr('pearson')
new_corr = corr.abs()
new_corr.loc[:,:] = np.tril(new_corr, k=-1) # below main lower triangle of an array
new_corr = new_corr.stack().to_frame('correlation').reset_index().sort_values(by='correlation', ascending=False)
new_corr[new_corr['correlation']>0.6]
```

# Para tratar los Nulls es MUY CLAVE ver el archivo 02_Tratamiento_Correlaciones_Outliers_nulls... Ahí están todas las opciones y métodos para poder tratar nulls y outliers!!!

La función `.fillna()` permite rellenar nos nulls:
```python
list_vars = list(set(list_var_continuous)-set(['revol_util']))
pd_loan_train[list_vars] = pd_loan_train[list_vars].fillna(-99)
pd_loan_test[list_vars] = pd_loan_test[list_vars].fillna(-99)
```

## Se puede usar KNN para rellenar tmb!
* Se usa para rellenar los Nulls con los valores de la row más parecida al resto de los datos de la row que se tiene
* KNN agrupa en base a Euclidian Distance entre los individuos (rows) de un DataSet
* OJO, muchas veces no es necesario usar un algorítmo tan complejo para fillear los nulls... Hay que ver que es mejor y más eficiente!


# Exploración de variables categóricas
* Tmb ver en el archivo 02_Tratamiento_Correlaciones_Outliers_nulls... Es importante
* Como no se puede ver la corr, se puede usar la función de Pandas `pd.crosstab()`, q está muy buena para ver como se cruzan las variables categóricas más importantes!
* No hace falta hacerlo con tooodas, sino para las variables que más se han diferenciado o resaltado en el gráfico descriptivo que vimos arriba!


### V de Cramer:

LO uso para ver un índice similar al de correlacion en matrices como esta de pd.crosstab(), osea que **Puedo ver la correlación entre dos variables categóricas! Si es cercano  a 1, va a haber correlación!**

* Si saco la V de Cramer para todas las variables categóricas, tmb la voy a poder dibujar como un HeatMap! Estaría muy interesante ver como hacerlo para la práctica!

### No olvidar, si rellené en el DataSet de Train, relleno igual en el DataSet de Test!!!


# Codificación de variables categóricas:

Siempre que se puedan cambiar categorías a números, en variables categóricas ordinales x ej, hacerlo, mejor.

Las categóricas pueden ser:
* Ordinales (meses x ejemplo)
* Nominales

## 1. One Hot Encoding

Codificar variables categóricas

La idea es SIEMPRE PONER N-1 CATEGORÍAS. Osea, si hay 5, la idea no es hacer un OHE de las 5, sino de las 4. Porque la quinta daría correlación! **LA función de SKL devuelve n-1 variables**. La última variable sería el ELSE, osea si no es nada de eso, es lo otro!

### En TREES NO HACE FALTA Q SEA N-1 PERO EN REGRESIONES X EJ SI, pq penalizan correlaciones!

Yo se que valor de la columna va a ser!

### Problema del OHE: Agrego alta Dimensionalidad al modelo! Solo para cuando hay POCAS categorías!

## 2. Ordinal Encoding
Lo puedo usar para codificar las variables Ordinales!


## 3. Target Encoding

Este método es muy bueno pq no crea muchas más columnas, sino que genera un código con el algorítmo en base a la variable objetivo!!!

(diferente al label encoding, es mejor!)

Este es de la libería categorialencoding, SKL tiene las de OneHotEncoding y OrdinalEncoding

Creamos **UNA columna en vez de tantas dimensiones como categorías...** 


## Hay más encodings, como el Frequency por ejemplo...

# Tanto One Hot Encoding como Target Encoding deben aplicarse primero al Train y aplicarlo dsps al Test!


# .fit se aplica en el Train y .transform en el test!!! Esto es CLAVE. No deberíamos usar .fit_transform en principio siempre q tengamos Train y Test!
Esto es pq el `.fit()` va a entrenar el modelo con el Train, va a guardar todo lo del Train y el `.transform()` se lo aplico al Test para q haga los mismos ajustes allí!





# <font color='deepskyblue'> APLICAR NOTEBOOK 1 Y 2 A LA PRÁCTICA! </font>


